In [1]:
%load_ext autoreload

In [25]:
import pandas as pd
from pathlib import Path
from typing import List, Dict
import darpinstances.instance
from darpinstances.db import db
import plotly.express as px

In [3]:
# darp_dir_path = Path(r"C:/Google Drive/AIC Experiment Data/DARP")
itsc_dir_path = Path(r"C:/Google Drive/AIC Experiment Data/DARP/ITSC_instance_paper/Instances")
# itsc_dir_path = Path(r"D:/Google Drive/AIC Experiment Data/DARP/ITSC_instance_paper/Instances")

area_dirs  = [
    'NYC',
    'Manhattan',
    'Chicago',
    'DC'
]

# Area table

In [30]:
def get_data_for_dir(area_dir: str) -> Dict:
    area_dir_path = itsc_dir_path / Path(area_dir)
    map_dir = area_dir_path / Path('map')
    node_path = map_dir / Path('nodes.csv')
    edge_path = map_dir / Path('edges.csv')

    nodes = pd.read_csv(node_path)
    edges = pd.read_csv(edge_path)

    area_data = {
        'area': area_dir,
        'node_count': len(nodes),
        'edge_count': len(edges)
    }


    # request count
    config = darpinstances.instance.load_instance_config(str(next(area_dir_path.rglob('config.yaml'))))
    demand_sets = config['demand']['dataset']
    area_id = config['area_id']
    # SELECT count(1) FROM demand
    #     WHERE
    #         dataset IN ({','.join((str(s) for s in demand_sets))})
    #         AND origin_time BETWEEN '2022-04-05 18:00:00' AND '2022-04-05 18:59:59'
    sql = f"""
    SELECT count(1) FROM demand
    JOIN trip_locations
        on demand.id = trip_locations.request_id
        AND dataset IN ({','.join((str(s) for s in demand_sets))})
        AND origin_time BETWEEN '2022-04-05 18:00:00' AND '2022-04-05 18:59:59'
    JOIN nodes origin_nodes
        ON origin_nodes.id = trip_locations.origin
    JOIN areas
        ON areas.id = {area_id}
        AND st_within(origin_nodes.geom, areas.geom)
    """
    req_count = db.execute_count_query(sql)
    area_data['Requests per hour'] = req_count

    # area
    sql = f"""
    SELECT st_area(st_transform(geom, 32618)) AS area
        FROM areas
        WHERE id = {area_id}
    """
    area = db.execute_count_query(sql)
    area_data['Area [km2]'] = area / 1000000

    return area_data

In [33]:
area_data_list = []
for area_dir in area_dirs:
    area_data = get_data_for_dir(area_dir)
    area_data_list.append(area_data)
area_data = pd.DataFrame(area_data_list)
area_data.rename(
    columns={
        'area': 'Area',
        'node_count': 'Node Count',
        'edge_count': 'Edge Count'
    },
    inplace=True
)
area_data

17:09:07 [INFO] Loading instance config from C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\NYC\instances\start_18-00\duration_005_min\max_delay_03_min\config.yaml
17:09:07 [INFO] Loading instance config from C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\Manhattan\instances\start_18-00\duration_005_min\max_delay_03_min\config.yaml
17:09:08 [INFO] Loading instance config from C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\Chicago\instances\start_18-00\duration_005_min\max_delay_03_min\config.yaml
17:09:08 [INFO] Loading instance config from C:\Google Drive\AIC Experiment Data\DARP\ITSC_instance_paper\Instances\DC\instances\start_18-00\duration_005_min\max_delay_03_min\config.yaml


,Area,Node Count,Edge Count,Requests per hour,Area [km2]
0,NYC,113411,281278,40294,1508.106916
1,Manhattan,6382,13455,23310,87.344297
2,Chicago,152653,413830,1138,1004.421323
3,DC,33230,84788,660,181.038363


In [34]:
area_data['Req./hour and km2'] = area_data['Requests per hour'] / area_data['Area [km2]']
area_data

,Area,Node Count,Edge Count,Requests per hour,Area [km2],Req./hour and km2
0,NYC,113411,281278,40294,1508.106916,26.718265
1,Manhattan,6382,13455,23310,87.344297,266.874894
2,Chicago,152653,413830,1138,1004.421323,1.132991
3,DC,33230,84788,660,181.038363,3.645636


In [10]:
s = area_data.style.format(escape='latex', precision=0)
s = s.hide(axis='index')
s = s.hide(['Requests per hour'], axis='columns')
print(s.to_latex())

NameError: name 'area_data' is not defined

# Demand density histogram

In [42]:
areas =[
    dict(name = 'NYC', id = 12, dataset_ids = [2, 3, 4, 5], srid = 32618, zone_types=[2]),
    dict(name = 'Manhattan', id = 4, dataset_ids = [2, 3, 4, 5], srid = 32618, zone_types=[2]),
    dict(name = 'Chicago', id = 19, dataset_ids = [1], srid = 26916, zone_types=[0, 1]),
    dict(name = 'DC', id = 22, dataset_ids = [7], srid = 32618, zone_types=[4])
]

def get_data_for_area(id: int, dataset_ids: List[int], srid: int) -> pd.DataFrame:
    sql = f"""
    SELECT
        zones.id,
        min(zones.name) AS name,
        count(demand.id) AS demand_count,
        st_area(st_transform(min(zones.geom), {srid})) AS area
    FROM areas
        JOIN zones ON areas.id = {id} AND st_within(zones.geom, areas.geom)
        JOIN demand ON
            demand.dataset IN ({','.join((str(id) for id in dataset_ids))})
            AND zones.id = demand.origin
            AND demand.origin_time BETWEEN '2022-04-05 00:00:00' AND '2022-04-05 23:59:59'
    GROUP BY zones.id
    """
    return db.execute_query_to_pandas(sql)



In [21]:
df = pd.DataFrame()
for area in areas:
    print(f"Importing data for {area['name']}")
    df_area = get_data_for_area(area['id'], area['dataset_ids'], area['srid'])
    df_area['area_name'] = area['name']
    df = pd.concat([df, df_area])
df

Importing data for NYC
Importing data for Manhattan
Importing data for Chicago
Importing data for DC


,id,name,demand_count,area,area_name
0,1,Newark Airport,25,7.338068e+06,NYC
1,3,Allerton/Pelham Gardens,1110,2.942011e+06,NYC
2,4,Alphabet City,1720,7.449769e+05,NYC
3,5,Arden Heights,163,4.680435e+06,NYC
4,6,Arrochar/Fort Wadsworth,293,3.800484e+06,NYC
...,...,...,...,...,...
1714,816865,1900 - 1999 BLOCK OF MASSACHUSETTS AVENUE SE,4,1.221537e+05,DC
1715,816873,1 - 49 BLOCK OF SUTTON SQUAE SW,8,4.172806e+03,DC
1716,816883,525 - 619 BLOCK OF WATER STREET SW,2,7.364737e+03,DC
1717,816888,1800 - 2099 BLOCK OF ASH ROAD SW,2,7.267198e+04,DC


In [35]:
df['demand_per_km2'] = df['demand_count'] * 1e6 / df['area']
df

,id,name,demand_count,area,area_name,demand_per_km2
127,132,JFK Airport,14923,1.820626e+07,NYC,819.663023
156,161,Midtown Center,14031,6.739298e+05,NYC,20819.674880
35,161,Midtown Center,14031,6.739298e+05,Manhattan,20819.674880
133,138,LaGuardia Airport,13069,2.927214e+06,NYC,4464.655199
231,237,Upper East Side South,12421,8.994033e+05,NYC,13810.267674
...,...,...,...,...,...,...
17,18,MONTCLARE,1,2.560172e+06,Chicago,0.390599
119,17031410800,None,1,2.490498e+05,Chicago,4.015262
105,110,Great Kills Park,1,2.172192e+06,NYC,0.460365
86,17031063400,None,1,1.587256e+05,Chicago,6.300182


In [40]:
fig = px.histogram(
    df,
    'demand_per_km2',
    nbins=20,
    facet_col='area_name',
    histnorm='percent'
)
fig

In [37]:
fig_nyc = px.histogram(
    df[df['area_name'] == 'NYC'],
    'demand_per_km2',
    nbins=20
)
fig_nyc

In [39]:
df.sort_values('demand_per_km2', inplace=True, ascending=False)
df

,id,name,demand_count,area,area_name,demand_per_km2
82,800713,1900 - 1999 BLOCK OF CONNECTICUT AVENUE NW,832,6.362633e+03,DC,130763.482471
1690,816675,0 - 0 BLOCK OF COLUMBUS CIRCLE NE,1414,1.999868e+04,DC,70704.683213
101,800876,1 - 99 BLOCK OF MASSACHUSETTS AVENUE NE,316,6.970222e+03,DC,45335.712157
1353,812637,900 - 999 BLOCK OF MASSACHUSETTS AVENUE NW,332,7.712457e+03,DC,43047.240947
8,800098,400 - 499 BLOCK OF NEW JERSEY AVENUE NW,330,8.936785e+03,DC,36926.032723
...,...,...,...,...,...,...
61,64,CLEARING,4,6.604980e+06,Chicago,0.605604
50,51,SOUTH DEERING,17,2.820138e+07,Chicago,0.602807
69,72,BEVERLY,4,8.241752e+06,Chicago,0.485334
105,110,Great Kills Park,1,2.172192e+06,NYC,0.460365


# Trip length histogram

In [48]:
def get_trip_length_for_area(dataset_ids: List[int], zone_types: List[int], srid: int):
    sql = f"""
    SELECT st_distance(st_transform(or_zones.geom, {srid}), st_transform(de_zones.geom, {srid})) AS distance

    FROM demand
    JOIN zones or_zones ON
        demand.dataset IN ({','.join((str(id) for id in dataset_ids))})
        AND demand.origin_time BETWEEN '2022-04-05 00:00:00' AND '2022-04-05 23:59:59'
        AND or_zones.id = demand.origin
        AND or_zones.type IN ({','.join((str(id) for id in zone_types))})
    JOIN zones de_zones ON de_zones.type IN ({','.join((str(id) for id in zone_types))}) AND de_zones.id = demand.destination
    """

    return db.execute_query_to_pandas(sql)

In [ ]:
df_distances = pd.DataFrame()
for area in areas:
    print(f"Importing data for {area['name']}")
    df_area = get_trip_length_for_area(area['dataset_ids'], area['zone_types'], area['srid'])
    df_area['area_name'] = area['name']
    df_distances = pd.concat([df_distances, df_area])
df_distances

In [45]:
df_distances['dist_km'] = df_distances['distance'] / 1000

In [47]:
dist_fig = px.histogram(
    df_distances,
    'dist_km',
    nbins=20,
    facet_col='area_name',
    histnorm='percent'
)
dist_fig